# Web Tree 구축

In [1]:
import os
import json
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def get_links_and_text(driver, url, root_url):
    """ 주어진 URL에서 내부 링크와 페이지의 텍스트 콘텐츠를 가져옴 """
    try:
        driver.get(url)
        time.sleep(2)  # 페이지 로딩 대기

        driver.execute_script("window.localStorage.clear();")  # 로컬 스토리지 삭제
        driver.execute_script("window.sessionStorage.clear();")  # 세션 스토리지 삭제
        driver.delete_all_cookies()  # 쿠키 삭제
        # print("✅ 캐시 및 쿠키 삭제 완료")

        for i in range(2):
            driver.refresh()
            # print("✅ 새로고침 완료")

        # 모든 내부 링크 가져오기
        links = set()
        elements = driver.find_elements(By.TAG_NAME, "a")
        for elem in elements:
            link = elem.get_attribute("href")
            if link and link.startswith(root_url):  # 내부 링크만 저장
                links.add(link)

        # 페이지의 텍스트 콘텐츠 가져오기
        page_content = driver.find_element(By.TAG_NAME, "body").text

        return links, page_content.strip()
    
    except Exception as e:
        print(f"Error while accessing {url}: {e}")
        return set(), ""

def crawl_website(root_name, root_url, max_depth=2, save_path="./"):
    """ 특정 사이트를 시작으로 내부 링크를 크롤링 (Depth별로 저장) 및 페이지 텍스트 저장 """
    options = Options()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    site_structure = {}  # Depth별 URL 저장
    page_contents = {}  # URL별 텍스트 저장
    visited = {}  # 방문한 URL 저장 (url: depth)

    def crawl(url, depth):
        if depth > max_depth or url in visited:
            return
        visited[url] = depth

        print(f"Crawling: {url} (Depth: {depth})")
        child_links, page_text = get_links_and_text(driver, url, root_url)

        # Depth별로 URL 저장
        if depth not in site_structure:
            site_structure[depth] = []
        site_structure[depth].append(url)

        # URL별 텍스트 저장
        page_contents[url] = page_text

        for link in child_links:
            crawl(link, depth + 1)

    crawl(root_url, 0)
    driver.quit()

    # JSON 파일 저장 (사이트 구조)
    with open(f"{save_path}/{root_name}_site_structure.json", "w", encoding="utf-8") as f:
        json.dump(site_structure, f, indent=4, ensure_ascii=False)

    # JSON 파일 저장 (페이지 콘텐츠)
    with open(f"{save_path}/{root_name}_page_contents.json", "w", encoding="utf-8") as f:
        json.dump(page_contents, f, indent=4, ensure_ascii=False)

    return site_structure, page_contents

In [3]:
root_name = "현대자동차그룹"
root_url = "https://audit.hyundai.com/"

save_path = "../data"
os.makedirs(save_path, exist_ok=True)
site_structure, page_contents = crawl_website(root_name, root_url, max_depth=6, save_path=save_path)

print(f"✅ {save_path}/{root_name}site_structure.json 저장 완료")
print(f"✅ {save_path}/{root_name}page_contents.json 저장 완료")

Crawling: https://audit.hyundai.com/ (Depth: 0)
Crawling: https://audit.hyundai.com/philosophy (Depth: 1)
Crawling: https://audit.hyundai.com/philosophy/23 (Depth: 2)
Crawling: https://audit.hyundai.com/progress (Depth: 3)
Crawling: https://audit.hyundai.com/site-map (Depth: 4)
Crawling: https://audit.hyundai.com/privacy-policy (Depth: 5)
Crawling: https://audit.hyundai.com/privacy-policy#contentPrivacy (Depth: 6)
Crawling: https://audit.hyundai.com/report/confirm-process (Depth: 6)
Crawling: https://audit.hyundai.com/report/guideline (Depth: 6)
Crawling: https://audit.hyundai.com/ethics-standard (Depth: 6)
Crawling: https://audit.hyundai.com/report/create (Depth: 6)
Crawling: https://audit.hyundai.com/site-map#contentSitemap (Depth: 5)
Crawling: https://audit.hyundai.com/progress#contentEthical (Depth: 4)
Crawling: https://audit.hyundai.com/philosophy/21 (Depth: 3)
Crawling: https://audit.hyundai.com/philosophy/21#contentPhilosophy (Depth: 4)
Crawling: https://audit.hyundai.com/philos

In [ ]:
# root_name = "남양유업"
# root_url = "https://company.namyangi.com/"

# save_path = "../data"
# os.makedirs(save_path, exist_ok=True)
# site_structure, page_contents = crawl_website(root_name, root_url, max_depth=3, save_path=save_path)

# print(f"✅ {save_path}/{root_name}site_structure.json 저장 완료")
# print(f"✅ {save_path}/{root_name}page_contents.json 저장 완료")

# RAG

## Document

In [4]:
import json

from uuid import uuid4
from langchain.docstore.document import Document

In [5]:
def load_documents(json_path, company_name):
    # 저장된 page_contents.json 파일 읽기
    with open(json_path, "r", encoding="utf-8") as f:
        page_contents = json.load(f)

    # Document 객체 리스트 생성
    documents = [
        Document(
            page_content=text,
            metadata={"company_name" : company_name, "url": url},
            id=uuid4()
        )
        for url, text in page_contents.items()
    ]

    return documents

In [6]:
data_path = "../data"
target_name = "현대자동차그룹"

target_file_path = f"{data_path}/{target_name}_page_contents.json"
documents = load_documents(target_file_path, company_name=target_name)
print(len(documents))

20


## Vector DB

Reference
- [https://wikidocs.net/234016](https://wikidocs.net/234016)

In [7]:
import os
import faiss

from datetime import datetime

from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

from langchain_openai import OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from dotenv import load_dotenv
load_dotenv("../keys.env")
OPENAI_API_KEY = os.getenv("GRAVY_LAB_OPENAI")

In [8]:
def load_embed_model(provider, model_name, api_key=None, model_kwargs=None, encode_kwargs=None):
    if provider == "openai":
        model = OpenAIEmbeddings(model=model_name, api_key=api_key)
    elif provider == "huggingface":
        model = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

    return model

def create_vector_db(index_type, embed_model):
    sample_query = "Hello_World!"

    if index_type == "L2": ## L2 Distance(Euclidean Distance)
        index = faiss.IndexFlatL2(len(embed_model.embed_query(sample_query)))

    elif index_type == "IP": ## Inner Product
        index = faiss.IndexFlatIP(len(embed_model.embed_query(sample_query)))

    elif index_type == "HNSW":## ANN -> Hierarchical Navigable Small World
        index = faiss.IndexHNSWFlat(len(embed_model.embed_query(sample_query)))

    vector_db = FAISS(
        embedding_function=embed_model,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )

    return vector_db

In [9]:
def save_vector_db(vector_db, save_path):
    vector_db.save_local(save_path)

def load_vector_db(index_file_path, embed_model):
    vector_db = FAISS.load_local(index_file_path, embeddings=embed_model, allow_dangerous_deserialization=True)

    return vector_db

In [10]:
now = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
save_path = f"../indexes/{now}"
os.makedirs(save_path, exist_ok=True)

In [11]:
provider = "huggingface"
model_name = "intfloat/multilingual-e5-large-instruct"
model_kwargs = {"device" : "cuda"}
encode_kwargs = {"normalize_embeddings": True}
index_type = "IP" ## "L2", "IP", "HNSW"

embed_model = load_embed_model(provider,
                               model_name,
                               api_key=OPENAI_API_KEY,
                               model_kwargs=model_kwargs, 
                               encode_kwargs=encode_kwargs)

vector_db = create_vector_db(index_type, embed_model)
vector_db.add_documents(documents)
vector_db.save_local(save_path)

In [12]:
retriever = vector_db.as_retriever()

In [15]:
results = retriever.invoke("기업의 비전에 대해 알려주세요.",
                           search_type="mmr",
                           search_kwargs={"k" : 5, "fetch_k" : 10, "lambda_mult" : 0.5})

for idx, result in enumerate(results):
    print("=" * 100)
    print(f"Result {idx:>04}")
    print(f"{result.metadata['url']}\n{result.page_content}\n\n")

Result 0000
https://audit.hyundai.com/philosophy/23#contentPhilosophy
기업이념
제보하기
기업이념체계
현대자동차그룹은 국민행복과 국가발전을 위해 ‘모범적인 기업’으로서의 역할에 더욱 책임을 다하겠습니다.
기업이념
기업이념체계
현대자동차그룹은 창립 이래 우리만의 고유한 정신과 가치를 계승하고 실천함으로써 지속 성장을 이루어 왔습니다. 이렇듯 현대자동차그룹이 추구하고 지켜나가야 할 가치와 변화의 방향, 그리고 미래상을 유기적으로 구성한 ‘경영이념체계’는 또 하나의 역사를 만들어 가는 구성원 모두의 목표이자 구심점입니다. 경영철학, 비전, 핵심가치를 이해관계자들과 공유함으로써, 구체적인 실천을 약속합니다.
경영철학
핵심가치
비전
비전
Together for a better future
더 나은 미래를 향한 동행
현대자동차그룹의 비전 ‘더 나은 미래를 향한 동행’은 기업의 외형적 성장을 넘어 최상의 만족을 고객에게 전달하고자 하는 굳은 의지를 담았습니다. 자동차, 철강, 건설을 중심으로 다양한 부문에서 시너지를 창출하는 현대자동차그룹만의 순환형 기업구조는 우리 생활 곳곳에 혁신을 일으키며 새로운 가치를 창조하고 있습니다. 인간존중과 환경친화적 경영을 기본 원칙으로 삼아 글로벌 선도 그룹의 위상에 맞게 발전해나갈 것 입니다.
자동차에서 삶의 동반자로
Lifetime partner in automobiles and beyond
인간중심적이고 환경친화적인 혁신 기술과 포괄적 서비스를 기반으로 최상의 이동성을 구현하여 삶을 더욱 편리하고 즐겁게 영위할 수 있는 새로운 공간을 제공합니다.
새로운 철강 시대의 리더
Leading the new era of steel
친환경, 자원순환형 기업으로서 대내외 유기적 협력관계를 기반으로 고도화된 제품 및 서비스를 제공하고 최상의 글로벌 경쟁력을 구현하여 새로운 철강시대를 선도합니다.
함께 내일을 창조하는 기업
We Build Tomorrow
글로벌 종합엔지니어링 기업으로서 유관산업과

In [16]:
results = retriever.invoke("기업의 비전에 대해 알려주세요.",
                           search_type="similarity_score_threshold",
                           search_kwargs={"score_threshold" : 0.55})

for idx, result in enumerate(results):
    print("=" * 100)
    print(f"Result {idx:>04}")
    print(f"{result.metadata['url']}\n{result.page_content}\n\n")

Result 0000
https://audit.hyundai.com/philosophy/23#contentPhilosophy
기업이념
제보하기
기업이념체계
현대자동차그룹은 국민행복과 국가발전을 위해 ‘모범적인 기업’으로서의 역할에 더욱 책임을 다하겠습니다.
기업이념
기업이념체계
현대자동차그룹은 창립 이래 우리만의 고유한 정신과 가치를 계승하고 실천함으로써 지속 성장을 이루어 왔습니다. 이렇듯 현대자동차그룹이 추구하고 지켜나가야 할 가치와 변화의 방향, 그리고 미래상을 유기적으로 구성한 ‘경영이념체계’는 또 하나의 역사를 만들어 가는 구성원 모두의 목표이자 구심점입니다. 경영철학, 비전, 핵심가치를 이해관계자들과 공유함으로써, 구체적인 실천을 약속합니다.
경영철학
핵심가치
비전
비전
Together for a better future
더 나은 미래를 향한 동행
현대자동차그룹의 비전 ‘더 나은 미래를 향한 동행’은 기업의 외형적 성장을 넘어 최상의 만족을 고객에게 전달하고자 하는 굳은 의지를 담았습니다. 자동차, 철강, 건설을 중심으로 다양한 부문에서 시너지를 창출하는 현대자동차그룹만의 순환형 기업구조는 우리 생활 곳곳에 혁신을 일으키며 새로운 가치를 창조하고 있습니다. 인간존중과 환경친화적 경영을 기본 원칙으로 삼아 글로벌 선도 그룹의 위상에 맞게 발전해나갈 것 입니다.
자동차에서 삶의 동반자로
Lifetime partner in automobiles and beyond
인간중심적이고 환경친화적인 혁신 기술과 포괄적 서비스를 기반으로 최상의 이동성을 구현하여 삶을 더욱 편리하고 즐겁게 영위할 수 있는 새로운 공간을 제공합니다.
새로운 철강 시대의 리더
Leading the new era of steel
친환경, 자원순환형 기업으로서 대내외 유기적 협력관계를 기반으로 고도화된 제품 및 서비스를 제공하고 최상의 글로벌 경쟁력을 구현하여 새로운 철강시대를 선도합니다.
함께 내일을 창조하는 기업
We Build Tomorrow
글로벌 종합엔지니어링 기업으로서 유관산업과

In [ ]:
results = vector_db.similarity_search_with_relevance_scores("현대자동차그룹의 비전에 대해 알려주세요.", k=10)

for document, score in results:
    print("=" * 100)
    print(f"{document.id}, {document.metadata['url']}")
    print(f"{score:.4f}")
    print(f"{document.page_content}\n\n")

31e135d3-b24e-4063-9e69-65c4f0a63fc0, https://audit.hyundai.com/philosophy/23#contentPhilosophy
0.3711
기업이념
제보하기
기업이념체계
현대자동차그룹은 국민행복과 국가발전을 위해 ‘모범적인 기업’으로서의 역할에 더욱 책임을 다하겠습니다.
기업이념
기업이념체계
현대자동차그룹은 창립 이래 우리만의 고유한 정신과 가치를 계승하고 실천함으로써 지속 성장을 이루어 왔습니다. 이렇듯 현대자동차그룹이 추구하고 지켜나가야 할 가치와 변화의 방향, 그리고 미래상을 유기적으로 구성한 ‘경영이념체계’는 또 하나의 역사를 만들어 가는 구성원 모두의 목표이자 구심점입니다. 경영철학, 비전, 핵심가치를 이해관계자들과 공유함으로써, 구체적인 실천을 약속합니다.
경영철학
핵심가치
비전
비전
Together for a better future
더 나은 미래를 향한 동행
현대자동차그룹의 비전 ‘더 나은 미래를 향한 동행’은 기업의 외형적 성장을 넘어 최상의 만족을 고객에게 전달하고자 하는 굳은 의지를 담았습니다. 자동차, 철강, 건설을 중심으로 다양한 부문에서 시너지를 창출하는 현대자동차그룹만의 순환형 기업구조는 우리 생활 곳곳에 혁신을 일으키며 새로운 가치를 창조하고 있습니다. 인간존중과 환경친화적 경영을 기본 원칙으로 삼아 글로벌 선도 그룹의 위상에 맞게 발전해나갈 것 입니다.
자동차에서 삶의 동반자로
Lifetime partner in automobiles and beyond
인간중심적이고 환경친화적인 혁신 기술과 포괄적 서비스를 기반으로 최상의 이동성을 구현하여 삶을 더욱 편리하고 즐겁게 영위할 수 있는 새로운 공간을 제공합니다.
새로운 철강 시대의 리더
Leading the new era of steel
친환경, 자원순환형 기업으로서 대내외 유기적 협력관계를 기반으로 고도화된 제품 및 서비스를 제공하고 최상의 글로벌 경쟁력을 구현하여 새로운 철강시대를 선도합니다.
함께 내일을 창조하는 기업
We Build

In [25]:
results = vector_db.similarity_search_with_relevance_scores("현대자동차그룹의 경영철학에 대해 알려주세요.", k=10)

for document, score in results:
    print("=" * 100)
    print(f"{document.id}, {document.metadata['url']}")
    print(f"{score:.4f}")
    print(f"{document.page_content}\n\n")

31e135d3-b24e-4063-9e69-65c4f0a63fc0, https://audit.hyundai.com/philosophy/23#contentPhilosophy
0.3570
기업이념
제보하기
기업이념체계
현대자동차그룹은 국민행복과 국가발전을 위해 ‘모범적인 기업’으로서의 역할에 더욱 책임을 다하겠습니다.
기업이념
기업이념체계
현대자동차그룹은 창립 이래 우리만의 고유한 정신과 가치를 계승하고 실천함으로써 지속 성장을 이루어 왔습니다. 이렇듯 현대자동차그룹이 추구하고 지켜나가야 할 가치와 변화의 방향, 그리고 미래상을 유기적으로 구성한 ‘경영이념체계’는 또 하나의 역사를 만들어 가는 구성원 모두의 목표이자 구심점입니다. 경영철학, 비전, 핵심가치를 이해관계자들과 공유함으로써, 구체적인 실천을 약속합니다.
경영철학
핵심가치
비전
비전
Together for a better future
더 나은 미래를 향한 동행
현대자동차그룹의 비전 ‘더 나은 미래를 향한 동행’은 기업의 외형적 성장을 넘어 최상의 만족을 고객에게 전달하고자 하는 굳은 의지를 담았습니다. 자동차, 철강, 건설을 중심으로 다양한 부문에서 시너지를 창출하는 현대자동차그룹만의 순환형 기업구조는 우리 생활 곳곳에 혁신을 일으키며 새로운 가치를 창조하고 있습니다. 인간존중과 환경친화적 경영을 기본 원칙으로 삼아 글로벌 선도 그룹의 위상에 맞게 발전해나갈 것 입니다.
자동차에서 삶의 동반자로
Lifetime partner in automobiles and beyond
인간중심적이고 환경친화적인 혁신 기술과 포괄적 서비스를 기반으로 최상의 이동성을 구현하여 삶을 더욱 편리하고 즐겁게 영위할 수 있는 새로운 공간을 제공합니다.
새로운 철강 시대의 리더
Leading the new era of steel
친환경, 자원순환형 기업으로서 대내외 유기적 협력관계를 기반으로 고도화된 제품 및 서비스를 제공하고 최상의 글로벌 경쟁력을 구현하여 새로운 철강시대를 선도합니다.
함께 내일을 창조하는 기업
We Build

In [26]:
results = vector_db.similarity_search_with_score("현대자동차그룹의 비전에 대해 알려주세요.", k=10)

for document, score in results:
    print("=" * 100)
    print(f"{document.id}, {document.metadata['url']}")
    print(f"{score:.4f}")
    print(f"{document.page_content}\n\n")

31e135d3-b24e-4063-9e69-65c4f0a63fc0, https://audit.hyundai.com/philosophy/23#contentPhilosophy
0.8894
기업이념
제보하기
기업이념체계
현대자동차그룹은 국민행복과 국가발전을 위해 ‘모범적인 기업’으로서의 역할에 더욱 책임을 다하겠습니다.
기업이념
기업이념체계
현대자동차그룹은 창립 이래 우리만의 고유한 정신과 가치를 계승하고 실천함으로써 지속 성장을 이루어 왔습니다. 이렇듯 현대자동차그룹이 추구하고 지켜나가야 할 가치와 변화의 방향, 그리고 미래상을 유기적으로 구성한 ‘경영이념체계’는 또 하나의 역사를 만들어 가는 구성원 모두의 목표이자 구심점입니다. 경영철학, 비전, 핵심가치를 이해관계자들과 공유함으로써, 구체적인 실천을 약속합니다.
경영철학
핵심가치
비전
비전
Together for a better future
더 나은 미래를 향한 동행
현대자동차그룹의 비전 ‘더 나은 미래를 향한 동행’은 기업의 외형적 성장을 넘어 최상의 만족을 고객에게 전달하고자 하는 굳은 의지를 담았습니다. 자동차, 철강, 건설을 중심으로 다양한 부문에서 시너지를 창출하는 현대자동차그룹만의 순환형 기업구조는 우리 생활 곳곳에 혁신을 일으키며 새로운 가치를 창조하고 있습니다. 인간존중과 환경친화적 경영을 기본 원칙으로 삼아 글로벌 선도 그룹의 위상에 맞게 발전해나갈 것 입니다.
자동차에서 삶의 동반자로
Lifetime partner in automobiles and beyond
인간중심적이고 환경친화적인 혁신 기술과 포괄적 서비스를 기반으로 최상의 이동성을 구현하여 삶을 더욱 편리하고 즐겁게 영위할 수 있는 새로운 공간을 제공합니다.
새로운 철강 시대의 리더
Leading the new era of steel
친환경, 자원순환형 기업으로서 대내외 유기적 협력관계를 기반으로 고도화된 제품 및 서비스를 제공하고 최상의 글로벌 경쟁력을 구현하여 새로운 철강시대를 선도합니다.
함께 내일을 창조하는 기업
We Build